<a href="https://colab.research.google.com/github/aakarshan-coding/AK-Megaslots/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd


titanic_dataset = pd.read_csv('Titanic-Dataset.csv')

X = titanic_dataset.drop(columns=['Survived','PassengerId','Name'])

Y = titanic_dataset['Survived']


for feature in X.columns.tolist():
  print(feature, X[feature].count())

Pclass 891
Sex 891
Age 714
SibSp 891
Parch 891
Ticket 891
Fare 891
Cabin 204
Embarked 889


In [24]:
X.head()

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,35.0,1,0,113803,53.1000,C123,S
4,3,male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
X['Ticket'].value_counts()

,count
Ticket,
347082,7
1601,7
CA. 2343,7
3101295,6
CA 2144,6
...,...
PC 17590,1
17463,1
330877,1


In [36]:
Y.value_counts()

,count
Survived,
0,549
1,342


In [40]:
from pandas.plotting import scatter_matrix

correlations = titanic_dataset[['Age','Pclass','SibSp','Parch','Fare','Survived']]

correlations.corr()

,Age,Pclass,SibSp,Parch,Fare,Survived
Age,1.000000,-0.369226,-0.308247,-0.189119,0.096067,-0.077221
Pclass,-0.369226,1.000000,0.083081,0.018443,-0.549500,-0.338481
SibSp,-0.308247,0.083081,1.000000,0.414838,0.159651,-0.035322
Parch,-0.189119,0.018443,0.414838,1.000000,0.216225,0.081629
Fare,0.096067,-0.549500,0.159651,0.216225,1.000000,0.257307
Survived,-0.077221,-0.338481,-0.035322,0.081629,0.257307,1.000000


In [51]:
import matplotlib.pyplot as plt

survived =titanic_dataset[(titanic_dataset['Survived'] == 1) & (titanic_dataset['Age'] < 25)].shape[0]
died = titanic_dataset[titanic_dataset['Survived'] == 0].shape[0]

total = titanic_dataset.shape[0]



print(survived/total * 100)

13.243546576879911


In [63]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import numpy as np



categorical_cols = ['Sex','Ticket','Embarked','Cabin']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot',OneHotEncoder(handle_unknown='ignore',sparse_output=False),categorical_cols)
    ],
    remainder='passthrough'
)

encoded_X = preprocessor.fit_transform(X)


ValueError: Shape of passed values is (891, 840), indices imply (9, 840)

In [84]:
X['Ticket'] = titanic_dataset['Ticket']
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Ticket
0,3,male,22.0,1,0,7.2500,NaN,S,A/5 21171
1,1,female,38.0,1,0,71.2833,C85,C,PC 17599
2,3,female,26.0,0,0,7.9250,NaN,S,STON/O2. 3101282
3,1,female,35.0,1,0,53.1000,C123,S,113803
4,3,male,35.0,0,0,8.0500,NaN,S,373450


In [85]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder, Normalizer
from sklearn.ensemble import RandomForestClassifier


categorical = ['Pclass','Sex','Ticket','Cabin','Embarked']
numerical = ['Age','SibSp','Parch','Fare']

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=.2,random_state=42)

preprocessor = ColumnTransformer([
    ('cat', Pipeline([
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
        ('imputer', KNNImputer(n_neighbors=5)),
        ('norm', Normalizer())
    ]), categorical),
    ('num', Pipeline([
        ('imputer', KNNImputer(n_neighbors=5)),
        ('norm', Normalizer())
    ]), numerical)
])



pipeline = Pipeline([
    ('pre',preprocessor),
    ('rf',RandomForestClassifier(n_estimators=100,random_state=42))
])

pipeline.fit(X_train,Y_train)

pipeline_pred = pipeline.predict(X_test)

score = pipeline.score(X_test, Y_test)

print(score)

0.8156424581005587
